In [6]:
# %load 
import torch as tch
from torch import nn
import torchaudio as ta
import torchaudio.transforms as tatx
import torchaudio.functional as tafn
from pathlib import Path
import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import Audio

from train_tools import find_lr, check_with_patient, UpdatingPlotlyLines, fit_step, evaluate
from torch.optim import Adam
from torch.nn import functional as nnfn

device = tch.device("cuda")

%load_ext autoreload
%autoreload 2 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from prep_we_sample import CommonVoiceDataset, get_segment, get_middle_segment, get_random_segment, get_middle_segment_jitter, PositiveSampleSet, NegativeSampleSet, MergePosNegSet

In [3]:
we_df = pd.read_csv('we_df.csv')
we_seg_df = pd.read_csv('wee_seg_16000.csv', index_col=0)

In [4]:
full_ds = CommonVoiceDataset('dataset/Common Voice Corpus 1/en/', 16000)
full_ds.use_df(full_ds.get_df('validated'))

we_ds = CommonVoiceDataset('dataset/Common Voice Corpus 1/en/', 16000)
we_ds.use_df(we_df)

In [26]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            nn.Conv1d(16, 16, 3), 
            nn.ReLU(), 
            nn.MaxPool1d(2), 

            #nn.Flatten(),
            nn.Linear(48, 1),
            nn.ReLU(), 

            #nn.ReLU(), 
            #nn.Linear(64, 1),
            #nn.Sigmoid(),
        )
    
    def forward(self, x):
        return tch.squeeze(self.conv(x))

m = Model()#.to('cuda')
optimiser = Adam(m.parameters(), 0.001)
m(get_middle_segment(we_ds, we_seg_df, 7, 400)[None, ...]).shape

torch.Size([16, 2])

In [8]:
i = 0
for k, v in m.state_dict().items():
    i += np.prod(v.shape)
i

54049

In [9]:
from torch.utils.data import DataLoader, Dataset, StackDataset


In [10]:
neg = NegativeSampleSet(full_ds, 400)
pos = PositiveSampleSet(we_ds, 400, we_seg_df, 0.5)
ds2 = MergePosNegSet(pos, neg)
dl = DataLoader(ds2, batch_size=64, shuffle=True)


In [139]:
find_lr(Model, dl, Adam, nnfn.binary_cross_entropy, starting_lr=1e-4)

FigureWidget({
    'data': [{'name': 'train_loss', 'type': 'scatter', 'uid': 'f882d2d8-aee3-43af-ab09-0fef0a7f031e', 'x': [], 'y': []},
             {'name': 'lr', 'type': 'scatter', 'uid': '373b06cc-fc07-423d-9f8f-4d6157b939e5', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

FigureWidget({
    'data': [{'name': 'train_loss', 'type': 'scatter', 'uid': 'f234bc99-5983-4ef9-a0d2-70c53649d051', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

In [11]:
from torch import optim 
from train_tools import find_lr, check_with_patient, UpdatingPlotlyLines, fit_step, evaluate, evaluate_v2

def fit(
        model: nn.Module, 
        train_dataloader: DataLoader, 
        val_dataloader:DataLoader, 
        optimiser: optim.Optimizer, 
        loss_fn, 
        epochs: int, 
        silent=True
    ):
    """
    reference implementation 
    """

    fig = UpdatingPlotlyLines('epoch', ['train_loss', 'acc'])
    fig.display()
    oop = check_with_patient(10)
    for epoch in range(epochs):
        train_loss = fit_step(model, train_dataloader, optimiser, loss_fn, silent=silent)
        y_pred, y = evaluate_v2(m, dl)

        acc = tch.sum(y == (y_pred>0.5))/len(y)
        

        fig.append(epoch=epoch, train_loss=train_loss, acc=acc.cpu())

        if oop(train_loss):
            return fig

    return fig 

m = Model().to('cuda')
o = Adam(m.parameters(), lr=0.002,)

fit(m, dl, dl, o, nnfn.binary_cross_entropy, 200 )



FigureWidget({
    'data': [{'name': 'train_loss', 'type': 'scatter', 'uid': '1d1b72b3-9ce0-4818-9027-75afd61e29cd', 'x': [], 'y': []},
             {'name': 'acc', 'type': 'scatter', 'uid': 'ccb09a32-2ba7-4b93-8361-d8a246a8d918', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

In [13]:
fit(m, dl, dl, o, nnfn.binary_cross_entropy, 200 )

FigureWidget({
    'data': [{'name': 'train_loss', 'type': 'scatter', 'uid': '4681040b-8bb5-4a64-9a9f-5b939c864e20', 'x': [], 'y': []},
             {'name': 'acc', 'type': 'scatter', 'uid': '964de9ca-69a8-467b-b9eb-d734f3d156e0', 'x': [], 'y': []}],
    'layout': {'template': '...'}
})

In [14]:
from train_tools import find_lr, check_with_patient, UpdatingPlotlyLines, fit_step, evaluate, evaluate_v2


In [118]:
y_pred, y = evaluate_v2(m, dl)

In [15]:
def evaluate_v3(model: nn.Module, dataloader: DataLoader):
    device = next(model.parameters()).device
    model.eval()


    fp_xs = []
    fn_xs = []

    fp_ys = []
    fn_ys = []



    with tch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)
            out = model(x)
            y_pred = out > 0.5

            y = y.to(bool)

            false_positive = y_pred & (~y)
            false_negative = (~y_pred) & y

            
            fp_xs.append(x[false_positive])
            fp_ys.append(out[false_positive])

            fn_xs.append(x[false_negative])
            fn_ys.append(out[false_negative])

        
    return tch.concat(fp_xs), tch.concat(fn_xs), tch.concat(fp_ys), tch.concat(fn_ys)
fp, fn, fp_ys, fn_ys = evaluate_v3(m, dl)

In [16]:
len(fn)

143

In [17]:
len(fp)

206

In [21]:
Audio(fp[0].cpu(), rate=16000)

In [22]:
Audio(fp[1].cpu(), rate=16000)

In [23]:
Audio(fp[2].cpu(), rate=16000)

In [24]:
Audio(fp[3].cpu(), rate=16000)

In [128]:
Audio(fp[4].cpu(), rate=16000)

In [18]:
Audio(fn[0].cpu(), rate=16000)

In [130]:
Audio(fn[1].cpu(), rate=16000)

In [19]:
Audio(fn[2].cpu(), rate=16000)

In [20]:
Audio(fn[3].cpu(), rate=16000)

In [116]:
fp.shape

torch.Size([206, 1, 6400])

In [117]:
evaluate(m, dl, nnfn.binary_cross_entropy)

0.1399318601384207

In [38]:
y_pred>0.5

tensor([False,  True, False,  ..., False,  True,  True], device='cuda:0')

In [119]:
tch.sum(y == (y_pred>0.5))/len(y)

tensor(0.9507, device='cuda:0')

In [120]:
tch.sum((y==1) & (y_pred>0.5))/tch.sum(y_pred>0.5)

tensor(0.9589, device='cuda:0')

In [121]:
tch.sum((y==0) & (y_pred>0.5))/tch.sum(y_pred>0.5)

tensor(0.0411, device='cuda:0')

In [122]:
tch.sum((y==1) & (y_pred<0.5))/tch.sum(y_pred<0.5)

tensor(0.0572, device='cuda:0')

In [45]:
tch.sum((y==0) & (y_pred<0.5))/tch.sum(y_pred<0.5)

tensor(0.9205, device='cuda:0')